# **Pneumonia detection using VGG16(Deep Learning)**

## **1. Import Libraries**

In [ ]:
from keras.layers import Input,Lambda,Dense,Flatten,Conv2D,MaxPooling2D
from keras.models import Model,Sequential
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

## **2.Pre-Processing**

In [ ]:
image_size=[224,224]
train_path='../input/labeled-chest-xray-images/chest_xray/train'
test_path='../input/labeled-chest-xray-images/chest_xray/test'

In [ ]:
train_datagen=ImageDataGenerator(rescale=1./255,shear_range=0.2,zoom_range=0.2,horizontal_flip=True)
test_datagen=ImageDataGenerator(rescale=1./255)

In [ ]:
train_data=train_datagen.flow_from_directory(train_path,target_size=(224,224),batch_size=32,class_mode='categorical')

In [ ]:
test_data=test_datagen.flow_from_directory(test_path,target_size=(224,224),batch_size=32,class_mode='categorical')

# **3.Models**

## **3.1 CNN**

In [ ]:
cnnmodel=Sequential()
cnnmodel.add(Conv2D(64,(3,3),activation='relu',input_shape=(224,224,3)))
cnnmodel.add(MaxPooling2D((2,2)))
cnnmodel.add(Conv2D(64,(3,3),activation='relu'))
cnnmodel.add(MaxPooling2D((2,2)))
cnnmodel.add(Flatten())
cnnmodel.add(Dense(32,activation='relu'))
cnnmodel.add(Dense(2,activation='softmax'))

In [ ]:
cnnmodel.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
res_cnnmodel=cnnmodel.fit(train_data,validation_data=test_data,epochs=25)

In [ ]:
# visualization of loss
plt.plot(res_cnnmodel.history['loss'],label='train loss')
plt.plot(res_cnnmodel.history['val_loss'],label='val loss')
plt.legend()
plt.show()

In [ ]:
pred_cnn=cnnmodel.predict(test_data)
pred_cnn=pred_cnn.argmax(axis=1)
pred_cnn

## **3.2 VGG16(Transfer Learning)**

In [ ]:
# VGG16 model
vgg=VGG16(input_shape=image_size+[3],weights='imagenet',include_top=False)

In [ ]:
# As all the layers are trained already
for layer in vgg.layers:
    layer.trainable=False

In [ ]:
folders=glob('../input/labeled-chest-xray-images/chest_xray/train/*') # to get the number of output classes

In [ ]:
x=Flatten()(vgg.output)
prediction=Dense(len(folders),activation='softmax')(x)
model=Model(inputs=vgg.input,outputs=prediction)

In [ ]:
model.summary()

In [ ]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics='accuracy')

In [ ]:
r=model.fit(train_data,validation_data=test_data,epochs=10)

In [ ]:
# visualization of loss
plt.plot(r.history['loss'],label='train loss')
plt.plot(r.history['val_loss'],label='val loss')
plt.legend()
plt.show()

In [ ]:
# prediction
pred_vgg=model.predict(test_data)
pred_vgg=pred_vgg.argmax(axis=1)
pred_vgg

## **4. Evaluation**

In [ ]:
train_accuracy=cnnmodel.evaluate(train_data)[1]
test_accuracy=cnnmodel.evaluate(test_data)[1]
print('Accuracy of CNN')
print('Train Accuracy: ',train_accuracy,'\nTest Accuracy: ',test_accuracy)

In [ ]:
train_accuracy1=model.evaluate(train_data)[1]
test_accuracy1=model.evaluate(test_data)[1]
print('Accuracy of VGG16')
print('Train Accuracy: ',train_accuracy1,'\nTest Accuracy: ',test_accuracy1)

# **THANK YOU**